# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [26]:
DATA_FOLDER = 'Data/ebola' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

# Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import glob  # finds all the pathnames matching a specified pattern
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.
from datetime import datetime, date, time
from dateutil.parser import parse

- Import the different reports in a DataFrame's and merge by country (Adding two different columns: report_date and country)
- Merge 3 DataFrame's in one
- For each country:
    - Daily average per month of new cases
    - Daily avg per month of deaths
  

# Task 1.1. Guinea

First, all the files on the guine_data folder have to be merged in a dataframe called `guinea_df`.

In [2]:
# Import Guinea data in one file
data_folder ='Data/ebola/guinea_data/'
allFiles = glob.glob(data_folder + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
guinea_df = pd.concat(list_)

print('Shape guinea_df:',guinea_df.shape)
print('Columns:', guinea_df.columns)

Shape guinea_df: (714, 25)
Columns: Index(['Beyla', 'Boffa', 'Conakry', 'Coyah', 'Dabola', 'Dalaba', 'Date',
       'Description', 'Dinguiraye', 'Dubreka', 'Forecariah', 'Gueckedou',
       'Kerouane', 'Kindia', 'Kissidougou', 'Kouroussa', 'Lola', 'Macenta',
       'Mzerekore', 'Nzerekore', 'Pita', 'Siguiri', 'Telimele', 'Totals',
       'Yomou'],
      dtype='object')


For our analysis, it is needed only the rows corresponding to the new cases and deaths and the columns of `Date`, `Description` and `Totals`. Let's check which raws we need to select by checking on the `Description` column.

In [18]:
# Switch the column order by name to have 'Date and 'Description' first. Used guinea_df.columns to know the columns
guinea_df = guinea_df[['Date','Description', 'Totals']]

guinea_df.head(5)

Date                        Description Totals
0  2014-08-04              New cases of suspects      5
1  2014-08-04             New cases of probables      0
2  2014-08-04             New cases of confirmed      4
3  2014-08-04  Total new cases registered so far      9
4  2014-08-04            Total cases of suspects     11

From the array above, we can identify different classes for new cases and new deaths. Let's select the raws of our interest: 
 - `Total new cases registered so far`
 - `New death registered today`
 - `New death registered`

As we can see, we should merge the two last classes of `Description` in order to compare both new cases and deaths. In addition, we are going to check the shape of boths dataframe to verify that we are not forgetting any information on the merging.

In [4]:
guinea_df_new = pd.concat([guinea_df[guinea_df.Description == 'Total new cases registered so far'],
                           guinea_df[guinea_df.Description  == 'New deaths registered'],
                           guinea_df[guinea_df.Description  == 'New deaths registered today']])

# Aggregate New deaths in only one category
guinea_df_new.Description = guinea_df_new.Description.replace('New deaths registered today', 'New deaths registered')

print('Shape guinea_df_new:', guinea_df_new.shape)

Shape guinea_df_new: (44, 3)


 Once the raws with all the relevant information for our analysis has been concatenated, we start the data cleaning 

In [8]:
# Fill nan values with a 0 if there is any value null
if guinea_df_new.Totals.isnull().any():
    guinea_df_new = guinea_df_new.fillna(value=0)

# Get the data type of each column
guinea_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 3 to 8
Data columns (total 3 columns):
Date           44 non-null object
Description    44 non-null object
Totals         44 non-null object
dtypes: object(3)
memory usage: 1.4+ KB


Let's transform the `Date` column in datetime format and the `Totals` column to numeric:

In [9]:
# Change to date format the Date column
guinea_df_new.Date = guinea_df_new.Date.apply(lambda d: pd.to_datetime(d))
guinea_df_new['Month'] = [date.month for date in guinea_df_new.Date] # Create a new column 'Month'
guinea_df_new = guinea_df_new[['Date','Month', 'Description', 'Totals']]

# Change to numeric the Totals
guinea_df_new['Totals'] = guinea_df_new['Totals'].apply(pd.to_numeric).astype(int)

guinea_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 3 to 8
Data columns (total 4 columns):
Date           44 non-null datetime64[ns]
Month          44 non-null int64
Description    44 non-null object
Totals         44 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 1.7+ KB


In order to compute the daily average monthly, some assumptions have to be taken in account:
- The data from the ebola dataset are from month 8 and 9. The only file containing data from month 10, as is taken on the first of the month, can be considered from the month before.


In [17]:
guinea_df_new.Month = guinea_df_new.Month.replace(10, 9)

# New cases
new_cases = guinea_df_new[guinea_df_new.Description == 'Total new cases registered so far']
new_cases_grouped = new_cases.groupby('Month').agg(np.sum)/30

# New deaths
new_deaths = guinea_df_new[guinea_df_new.Description == 'New deaths registered']
new_deaths_grouped = new_deaths.groupby('Month').agg(np.sum)/30

guinea_conc = pd.concat([new_cases_grouped, new_deaths_grouped], axis=1)
guinea_conc = guinea_conc.rename(columns={'Totals': 'avg_new_cases', 'Totals': 'avg_new_deaths'})
guinea_conc

avg_new_deaths  avg_new_deaths
Month                                
8                 4.3        0.566667
9                11.6        2.400000

# Task 1.2. Liberia 

In [19]:
# Import Liberia data in one file
data_folder ='Data/ebola/liberia_data/'
allFiles = glob.glob(data_folder + "/*.csv")
liberia_df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
liberia_df = pd.concat(list_)

print('Shape guinea_df:',guinea_df.shape)
print('Columns:', guinea_df.columns)

Shape guinea_df: (714, 3)
Columns: Index(['Date', 'Description', 'Totals'], dtype='object')


In [20]:
liberia_df = liberia_df[['Date','Variable', 'National']]
liberia_df.head(5)

Date                          Variable  National
0  6/16/2014               Specimens collected       1.0
1  6/16/2014     Specimens pending for testing       0.0
2  6/16/2014            Total specimens tested      28.0
3  6/16/2014             Newly reported deaths       2.0
4  6/16/2014  Total death/s in confirmed cases       8.0

In [88]:
#liberia_df.Variable.value_counts().head(20)

As we did in the guina dataset, we need to select the variables names related to the new cases and new deaths:
- Newly reported deaths
- New case/s (confirmed)
- New Case/s (Suspected)
- New Case/s (Probable)

In [37]:
# New cases 
liberia_df_new = pd.concat([liberia_df[liberia_df.Variable == 'New Case/s (Probable)'],
                         liberia_df[liberia_df.Variable  == 'New Case/s (Suspected)'],
                         liberia_df[liberia_df.Variable  == 'New case/s (confirmed)'],
                         liberia_df[liberia_df.Variable  == 'Newly reported deaths']])

print('Shape liberia_df_new:', liberia_df_new.shape)
#new_cases_l.set_index('Date', inplace=True)
#new_cases_l = new_cases_l.groupby(new_cases_l.index).sum()

# New deaths


Shape liberia_df_new: (400, 3)


In [52]:
# Fill nan values with a 0 if there is any value null
if liberia_df_new.National.isnull().any():
    liberia_df_new = liberia_df_new.fillna(value=0)

# Get the data type of each column
liberia_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 24 to 3
Data columns (total 4 columns):
Date        400 non-null datetime64[ns]
Month       400 non-null int64
Variable    400 non-null object
National    400 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 15.6+ KB


Now we need to change the format of the columns to treat them.

In [53]:
# Change to date format the Date column
liberia_df_new.Date = liberia_df_new.Date.apply(lambda d: pd.to_datetime(d))
liberia_df_new['Month'] = [date.month for date in liberia_df_new.Date] # Create a new column 'Month'
liberia_df_new = liberia_df_new[['Date','Month', 'Variable', 'National']]

# Change to numeric the Totals
liberia_df_new['National'] = liberia_df_new['National'].apply(pd.to_numeric).astype(int)

liberia_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 24 to 3
Data columns (total 4 columns):
Date        400 non-null datetime64[ns]
Month       400 non-null int64
Variable    400 non-null object
National    400 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 15.6+ KB


In [87]:
days_per_month = [15, 30, 30, 30, 30, 30]  # Do it proper

# New cases
new_cases_l = pd.concat([liberia_df_new[liberia_df_new.Variable == 'New Case/s (Probable)'],
                         liberia_df_new[liberia_df_new.Variable  == 'New Case/s (Suspected)'],
                         liberia_df_new[liberia_df_new.Variable  == 'New case/s (confirmed)']])

new_cases_l_grouped = new_cases_l.groupby('Month').agg(np.sum)
new_cases_l_grouped = new_cases_l_grouped.rename(columns={'National': 'avg_new_cases'})

# New deaths
new_deaths_l = liberia_df_new[liberia_df_new.Variable == 'Newly reported deaths']
new_deaths_l_grouped = new_deaths_l.groupby('Month').agg(np.sum)
new_deaths_l_grouped = new_deaths_l_grouped.rename(columns={'National': 'avg_new_deaths'})
liberia_conc = pd.concat([new_cases_l_grouped, new_deaths_l_grouped], axis=1)
liberia_conc.drop(12, inplace=True)

liberia_conc.avg_new_cases = liberia_conc.avg_new_cases/days_per_month
liberia_conc.avg_new_deaths = liberia_conc.avg_new_deaths/days_per_month

liberia_conc

avg_new_cases  avg_new_deaths
Month                               
6           2.666667        0.933333
7           3.133333        1.566667
8          11.166667        6.966667
9          51.066667       28.833333
10         37.966667       23.366667
11         13.233333        6.733333

# Sierra leone


In [135]:
# Import Guinea data in one file
data_folder ='Data/ebola/sl_data/'
allFiles = glob.glob(data_folder + "/*.csv")
print('Number of csv files:',len(allFiles))
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
sl_df = pd.concat(list_)

print('Shape sl_df:',sl_df.shape)
print('Columns:', sl_df.columns)

Number of csv files: 103
Shape sl_df: (3262, 27)
Columns: Index(['34 Military Hospital', 'Bo', 'Bo EMC', 'Bombali', 'Bonthe',
       'Hastings-F/Town', 'Kailahun', 'Kambia', 'Kenema', 'Kenema (IFRC)',
       'Kenema (KGH)', 'Koinadugu', 'Kono', 'Moyamba', 'National',
       'Police training School', 'Police traning School', 'Port Loko',
       'Pujehun', 'Tonkolili', 'Unnamed: 18', 'Western area',
       'Western area combined', 'Western area rural', 'Western area urban',
       'date', 'variable'],
      dtype='object')


In [136]:
# Switch the column order by name to have 'Date and 'Description' first. Used guinea_df.columns to know the columns
sl_df = sl_df[['date','variable', 'National']]
sl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3262 entries, 0 to 36
Data columns (total 3 columns):
date        3262 non-null object
variable    3262 non-null object
National    2339 non-null object
dtypes: object(3)
memory usage: 101.9+ KB


In [ ]:
# New cases 
sl_df_new = pd.concat([sl_df[sl_df.variable == 'New Case/s (Probable)'],
                         sl_df[sl_df.variable  == 'New Case/s (Suspected)'],
                         sl_df[sl_df.variable  == 'New case/s (confirmed)']])

In [139]:
# Fill nan values with a 0 if there is any value null
if sl_df.National.isnull().any():
    sl_df = sl_df.fillna(value=0)

# Get the data type of each column
liberia_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 24 to 3
Data columns (total 4 columns):
Date        400 non-null datetime64[ns]
Month       400 non-null int64
Variable    400 non-null object
National    400 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 15.6+ KB


In [138]:
# Change to date format the Date column
sl_df.date = sl_df.date.apply(lambda d: pd.to_datetime(d))
sl_df['Month'] = [date.month for date in sl_df.date] # Create a new column 'Month'
sl_df = sl_df[['date','Month', 'variable', 'National']]

# Change to numeric the Totals
#sl_df['National'] = sl_df['National'].apply(pd.to_numeric).astype(int)

sl_df

date  Month                variable   National
0  2014-08-12      8              population    6348350
1  2014-08-12      8             new_noncase          4
2  2014-08-12      8           new_suspected         10
3  2014-08-12      8            new_probable          1
4  2014-08-12      8           new_confirmed         11
5  2014-08-12      8             cum_noncase        657
6  2014-08-12      8           cum_suspected         46
7  2014-08-12      8            cum_probable         37
8  2014-08-12      8           cum_confirmed        717
9  2014-08-12      8         death_suspected          5
10 2014-08-12      8          death_probable         34
11 2014-08-12      8         death_confirmed        264
12 2014-08-12      8                     cfr       36.8
13 2014-08-12      8       etc_new_admission          0
14 2014-08-12      8  etc_currently_admitted          0
15 2014-08-12      8       etc_cum_admission          0
16 2014-08-12      8          etc_new_deaths          0
17 2014-08-12      8          etc_cum_deaths          0
18 2014-08-12      8      etc_new_discharges          0
19 2014-08-12      8      etc_cum_discharges          0
20 2014-08-12      8            cum_contacts       2838
21 2014-08-12      8  cum_completed_contacts        901
22 2014-08-12      8       contacts_followed       2255
23 2014-08-12      8            new_contacts        743
24 2014-08-12      8        contacts_healthy       1937
25 2014-08-12      8            contacts_ill         14
26 2014-08-12      8       contacts_not_seen        131
27 2014-08-12      8  new_completed_contacts         73
28 2014-08-12      8            percent_seen        87%
0  2014-08-13      8              population  6,348,350
..        ...    ...                     ...        ...
7  2014-12-13     12            cum_probable         79
8  2014-12-13     12           cum_confirmed       6638
9  2014-12-13     12         death_suspected          0
10 2014-12-13     12          death_probable          0
11 2014-12-13     12         death_confirmed          0
12 2014-12-13     12                     cfr          0
13 2014-12-13     12       etc_new_admission          0
14 2014-12-13     12  etc_currently_admitted          0
15 2014-12-13     12       etc_cum_admission          0
16 2014-12-13     12          etc_new_deaths          0
17 2014-12-13     12          etc_cum_deaths          0
18 2014-12-13     12      etc_new_discharges          0
19 2014-12-13     12      etc_cum_discharges          0
20 2014-12-13     12            cum_contacts          0
21 2014-12-13     12  cum_completed_contacts          0
22 2014-12-13     12       contacts_followed          0
23 2014-12-13     12            new_contacts          0
24 2014-12-13     12        contacts_healthy          0
25 2014-12-13     12            contacts_ill          0
26 2014-12-13     12       contacts_not_seen          0
27 2014-12-13     12  new_completed_contacts          0
28 2014-12-13     12            percent_seen          0
29 2014-12-13     12       total_lab_samples          0
30 2014-12-13     12          repeat_samples          0
31 2014-12-13     12             new_samples          0
32 2014-12-13     12            new_positive          0
33 2014-12-13     12            new_negative          0
34 2014-12-13     12                 pending          0
35 2014-12-13     12         positive_corpse          0
36 2014-12-13     12         negative_corpse          0

[3262 rows x 4 columns]

In [118]:
import matplotlib.pyplot as plt
%matplotlib inline
    
#sl_df.set_index('date', inplace=True)
#sl_df.National[sl_df.variable == 'etc_cum_deaths'].plot()


TypeError: Empty 'DataFrame': no numeric data to plot

In [ ]:
guinea_df_new = pd.concat([guinea_df[guinea_df.Description == 'Total new cases registered so far'],
                           guinea_df[guinea_df.Description  == 'New deaths registered'],
                           guinea_df[guinea_df.Description  == 'New deaths registered today']])

# Aggregate New deaths in only one category
guinea_df_new.Description = guinea_df_new.Description.replace('New deaths registered today', 'New deaths registered')

print('Shape guinea_df_new:', guinea_df_new.shape)

In [ ]:
# Fill nan values with a 0 if there is any value null
if guinea_df_new.Totals.isnull().any():
    guinea_df_new = guinea_df_new.fillna(value=0)

# Get the data type of each column
guinea_df_new.info()

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here